In [1]:
!pip install "tensorflow~=2.0"
!pip install "tensorflow-hub[make_image_classifier]~=0.6"

In [35]:
import os
import time

import tensorflow as tf
import numpy as np
from PIL import Image

In [3]:
!make_image_classifier --helpfull

2020-07-19 04:41:09.821527: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1

       USAGE: /usr/local/bin/make_image_classifier [flags]
flags:

absl.app:
  -?,--[no]help: show this help
    (default: 'false')
  --[no]helpfull: show full help
    (default: 'false')
  --[no]helpshort: show this help
    (default: 'false')
  --[no]helpxml: like --helpfull, but generates XML output
    (default: 'false')
  --[no]only_check_args: Set to true to validate args and exit.
    (default: 'false')
  --[no]pdb_post_mortem: Set to true to handle uncaught exceptions with PDB post
    mortem.
    (default: 'false')
  --profile_file: Dump profile information to a file (for python -m pstats).
    Implies --run_with_profiling.
  --[no]run_with_pdb: Set to true for PDB debug mode
    (default: 'false')
  --[no]run_with_profiling: Set to true for profiling the script. Execution will
    be slower, and the output format might change over 

In [5]:
!unzip ./dataset.zip
!rm -r ./dataset.zip

Archive:  ./dataset.zip
  inflating: nishikino_maki/1.jpg    
  inflating: nishikino_maki/10.jpg   
  inflating: nishikino_maki/11.jpg   
  inflating: nishikino_maki/12.jpg   
  inflating: nishikino_maki/13.jpg   
  inflating: nishikino_maki/14.jpg   
  inflating: nishikino_maki/15.jpg   
  inflating: nishikino_maki/16.jpg   
  inflating: nishikino_maki/17.jpg   
  inflating: nishikino_maki/18.jpg   
  inflating: nishikino_maki/19.jpg   
  inflating: nishikino_maki/2.jpg    
  inflating: nishikino_maki/20.jpg   
  inflating: nishikino_maki/21.jpg   
  inflating: nishikino_maki/22.jpg   
  inflating: nishikino_maki/23.jpg   
  inflating: nishikino_maki/24.jpg   
  inflating: nishikino_maki/25.jpg   
  inflating: nishikino_maki/26.jpg   
  inflating: nishikino_maki/27.jpg   
  inflating: nishikino_maki/28.jpg   
  inflating: nishikino_maki/29.jpg   
  inflating: nishikino_maki/3.jpg    
  inflating: nishikino_maki/30.jpg   
  inflating: nishikino_maki/4.jpg    
  inflating: nishikino_mak

In [30]:
!make_image_classifier \
  --image_dir dataset \
  --batch_size 15 \
  --tfhub_module https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4 \
  --image_size 224 \
  --saved_model_dir anime-model \
  --labels_output_file class_labels.txt \
  --tflite_output_file anime-tflite-model.tflite

2020-07-19 04:59:03.289391: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
I0719 04:59:05.264416 140611373709184 resolver.py:79] Using /tmp/tfhub_modules to cache modules.
2020-07-19 04:59:05.450814: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcuda.so.1
2020-07-19 04:59:05.454144: E tensorflow/stream_executor/cuda/cuda_driver.cc:313] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2020-07-19 04:59:05.454198: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (4d7938ca6515): /proc/driver/nvidia/version does not exist
2020-07-19 04:59:05.460377: I tensorflow/core/platform/profile_utils/cpu_utils.cc:102] CPU Frequency: 2300000000 Hz
2020-07-19 04:59:05.460647: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x2449100 initialized for platform Host (this

In [32]:
# Test with our own images!
ARGS_INPUT_MEAN_DEFAULT = 127

def load_labels(filename):
  with open(filename, 'r') as f:
    return [line.strip() for line in f.readlines()]

# First, load the TFLite model that we just created and allocate some tensors.
interpreter = tf.lite.Interpreter(model_path = "anime-tflite-model.tflite")
interpreter.allocate_tensors()

# Second, get input and output tensors.
input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# check the type of the input tensor
floating_model = input_details[0]['dtype'] == np.float32

# NxHxWxC, H:1, W:2
height = input_details[0]['shape'][1]
width = input_details[0]['shape'][2]
img = Image.open("31.jpg").resize((width, height))

# add N dim
input_data = np.expand_dims(img, axis=0)

if floating_model:
  input_data = (np.float32(input_data) - ARGS_INPUT_MEAN_DEFAULT) / ARGS_INPUT_MEAN_DEFAULT

interpreter.set_tensor(input_details[0]['index'], input_data)

start_time = time.time()
interpreter.invoke()
stop_time = time.time()

output_data = interpreter.get_tensor(output_details[0]['index'])
results = np.squeeze(output_data)

top_k = results.argsort()[-5:][::-1]
labels = load_labels('class_labels.txt')
  
for i in top_k:
  if floating_model:
    print('{:08.6f}: {}'.format(float(results[i]), labels[i]))
  else:
    print('{:08.6f}: {}'.format(float(results[i] / 255.0), labels[i]))

print('time: {:.3f}ms'.format((stop_time - start_time) * 1000))

0.566001: nishikino_maki
0.320130: sakurauchi_riko
0.096640: erza_scarlet
0.017230: takimoto_hifumi
time: 45.025ms


In [44]:
def recreate_labels():
  # 1) We use this in order to ignore any hidden files that might be here.
  labels = [folder for folder in os.listdir('datasets') if not folder.startswith('.')]
  
  # 2) Then, we output the contents of each folder name to a file.
  with open('labels.txt', 'w') as file:
    for label in labels:
      file.write(label)
      file.write('\n')

recreate_labels()

['dandellions', 'tulips', 'roses', 'lilies']
